In [3]:
from pymongo import MongoClient
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import pprint
import time
import json
from contextlib import contextmanager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException

In [4]:
client = MongoClient()
testing_db=client['testing']
testcoll=testing_db['testcol']

In [5]:
url_main='https://www.rottentomatoes.com/browse/dvd-streaming-all?minTomato=0&maxTomato=1&services=amazon;hbo_go;itunes;netflix_iw;vudu;amazon_prime;fandango_now&genres=1;2;4;5;6;8;9;10;11;13;18;14&sortBy=release'


In [6]:
class text_to_be_not_present_in_element(object):
    """ An expectation for checking if the given text is present in the
    specified element.
    locator, text
    """
    def __init__(self, locator, text_):
        self.locator = locator
        self.text = text_

    def __call__(self, driver):
        try :
            element_text = EC._find_element(driver, self.locator).text
            return not(self.text in element_text)
        except StaleElementReferenceException:
            return False

In [11]:
def get_urls(x):
    movie_links = []
    driver = webdriver.Chrome(executable_path='./src/chromedriver')
    driver.get(x)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    show_count = soup.find('span',attrs={'id':'showing-count'}).text
    total_count = int(show_count.split(' ')[3])
    
    elem = driver.find_element_by_xpath('//*[@id="show-more-btn"]/button')
    
    
    while(True):
            

        #check current count vs total count
        soup = BeautifulSoup(driver.page_source,"lxml")
        show_count = soup.find('span',attrs={'id':'showing-count'}).text

        print(show_count)

        current_count = int(show_count.split(' ')[1])

        #if it has finished clicking, break out of while loop
        if(current_count >= (total_count-1)):
                break

        else:
            #continue clicking
            try:
                elem.click()
                wait = WebDriverWait(driver, 90)
                #wait until mb-movies is not stale
                # elem = wait.until((float(driver.find_element_by_class_name('mb-movies').get_attribute("style").split('opacity: ')[1][0]) == 0.5))
                # elem = wait.until((float(driver.find_element_by_class_name('mb-movies').get_attribute("style").split('opacity: ')[1][0]) == 1))
                wait = wait.until(text_to_be_not_present_in_element((By.ID,'showing-count'),show_count))
            except ElementNotVisibleException:
                break
            except TimeoutException:
                break
        
    print('Scraping now')
        
    #record each movie title and its url inside dict
    soup = BeautifulSoup(driver.page_source,"lxml")
    movies = soup.find('div', {'class' :"mb-movies"})
    for movie in movies:
        url = movie.find('a',{'class' : "popoverTrigger"})['href']
        

        movie_links.append(url)
        
    driver.quit()
    
    

In [12]:
get_urls(url_main)

ValueError: invalid literal for int() with base 10: ''